In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import tf_utils
import time


np.random.seed(1)

y_hat = tf.constant(36, name = "y_hat")
y = tf.constant(39, name = "y")
loss = tf.Variable((y_hat - y)**2, name = "loss")

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    print(session.run(loss))

In [ ]:
def linear_function():
    np.random.seed(1)   
    x = np.random.randn(3, 1)
    w = np.random.randn(4, 3)
    b = np.random.randn(4, 1)
    
    y = tf.matmul(w, x) + b
    
    session = tf.Session()
    result = session.run(y)
    session.close()
    
    return result
print("result = " +  str(linear_function()))

In [ ]:
def sigmoid(z):
    x = tf.placeholder(tf.float32, name = "x")
    sigmoid = tf.sigmoid(x)
    with tf.Session() as sess:
        result = sess.run(sigmoid, feed_dict = {x:z})
    return result
print ("sigmoid(0) = " + str(sigmoid(0)))
print ("sigmoid(12) = " + str(sigmoid(12)))

In [ ]:
def one_hot_matrix(lables, C):
    C = tf.constant(C, name = "C")
    on_hot_matrix = tf.one_hot(indices = lables, depth = C, axis = 0)
    with tf.Session() as sess:
        one_hot = sess.run(on_hot_matrix)
    return one_hot
labels = np.array([1,2,3,0,2,1])
one_hot = one_hot_matrix(labels,C=4)
print(str(one_hot))

总结：
1. tf框架中，各种运算的定义仅仅是定义的是计算图，真正要出结果，要使用session.
2. tf.placeholder定义的变量只是一个空架子，在run含有这样的变量的式子时，需要使用feed_dict给值
3. tf.one_hot：indices表示向量第几个数是1，depth是分类数，也是矩阵的行数

In [ ]:
X_train_orig , Y_train_orig , X_test_orig , Y_test_orig , classes = tf_utils.load_dataset()

In [ ]:
index = 11
plt.imshow(X_train_orig[index])
print("Y = " + str(np.squeeze(Y_train_orig[:,index])))

In [ ]:
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T
X_train = X_train_flatten / 255
X_test = X_test_flatten / 255

Y_train = tf_utils.convert_to_one_hot(Y_train_orig,6)
Y_test = tf_utils.convert_to_one_hot(Y_test_orig, 6)
print("训练集样本数 = " + str(X_train.shape[1]))
print("测试集样本数 = " + str(X_test.shape[1]))
print("X_train.shape: " + str(X_train.shape))
print("Y_train.shape: " + str(Y_train.shape))
print("X_test.shape: " + str(X_test.shape))
print("Y_test.shape: " + str(Y_test.shape))

In [ ]:
def create_placeholders(n_x,n_y):
    X = tf.placeholder(tf.float32, [n_x, None], name = "X")
    Y = tf.placeholder(tf.float32, [n_y, None], name = "Y")
    return X, Y

In [ ]:
X, Y = create_placeholders(12288, 6)
print("X = " + str(X))
print("Y = " + str(Y))

In [ ]:
def initialize_parameters():
    tf.set_random_seed(1)
    w1 = tf.get_variable("W1", [25, 12288], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [25, 1], initializer = tf.zeros_initializer())
    w2 = tf.get_variable("W2", [12, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [12, 1], initializer = tf.zeros_initializer())
    w3 = tf.get_variable("w3", [6, 12], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [6, 1], initializer = tf.zeros_initializer())
    parameters = {
        "W1":w1,
        "b1":b1,
        "W2":w2,
        "b2":b2,
        "W3":w3,
        "b3":b3
    }
    
    return parameters

In [ ]:
tf.reset_default_graph() #用于清除默认图形堆栈并重置全局默认图形。 

with tf.Session() as sess:
    parameters = initialize_parameters()
    print("W1 = " + str(parameters["W1"]))
    print("b1 = " + str(parameters["b1"]))
    print("W2 = " + str(parameters["W2"]))
    print("b2 = " + str(parameters["b2"]))

In [ ]:
def forward_propagation(X,parameters):
    w1 = parameters["W1"]
    b1 = parameters["b1"]
    w2 = parameters["W2"]
    b2 = parameters["b2"]
    w3 = parameters["W3"]
    b3 = parameters["b3"]
    
    Z1 = tf.matmul(w1, X) + b1
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(w2, A1) + b2
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(w3, A2) + b3
    #A3 = tf.nn.relu(Z3)


        
    
    return Z3

In [ ]:
tf.reset_default_graph() #用于清除默认图形堆栈并重置全局默认图形。 
with tf.Session() as sess:
    X,Y = create_placeholders(12288,6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X,parameters)
    print("Z3 = " + str(Z3))

In [ ]:
def compute_cost(Z3,Y):
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    return cost

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    X,Y = create_placeholders(12288,6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X,parameters)
    cost = compute_cost(Z3,Y)
    print("cost = " + str(cost))

In [ ]:
def model(X_train,Y_train,X_test,Y_test,learning_rate=0.0001,num_epochs=1500,minibatch_size=32,print_cost=True,is_plot=True):
    ops.reset_default_graph()
    tf.set_random_seed(1)
    seed = 3
    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0]
    costs = []
    X,Y = create_placeholders(n_x, n_y)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0
            num_minibatches = int(m / minibatch_size)
            seed = seed + 1
            minibatches = tf_utils.random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _, minibatch_cost = sess.run([optimizer, cost], feed_dict = {X:minibatch_X, Y:minibatch_Y})
                epoch_cost = epoch_cost + minibatch_cost / num_minibatches  #还不懂，为什么要除以batch的数量
            
            if epoch % 5 == 0:
                costs.append(epoch_cost)
                if print_cost and epoch % 100 == 0:
                    print("epoch = {}\t|epoch_cost = {}".format(epoch, epoch_cost))
            
        if is_plot:
            plt.plot(np.squeeze(costs))
            plt.ylabel("cost")
            plt.xlabel("epoch")
            plt.title("Learning rate = " + str(learning_rate))
            plt.show()

                          
        parameters = sess.run(parameters)  #需要使用run取出最终的参数值
        corrent_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))  # tf.argmax沿着某个轴找最大值的索引，默认是0轴。
        accuracy = tf.reduce_mean(tf.cast(corrent_prediction, "float"))
                          
        print("accuracy_train:", accuracy.eval({X:X_train, Y:Y_train}))
        print("accuracy_test: ", accuracy.eval({X:X_test, Y:Y_test}))
        # accuracy.eval({X:X_train, Y:Y_train})等价于sess.run(accuracy, feed_dict = {X:X_train, Y:Y_train}).
        return parameters



准备工作：创建X、Y的占位符，只需指定特征维度，初始化参数
1.前向传播，获得Z3
2.计算loss
3.准备好优化器，反向传播用
4.创建session,优化loss：
   4.1 每个epoch前，准备好随机的小批量数据集
   4.2 关键代码：sess.run([optimizer, cost], feed_dict = {X:minibatch_X, Y:minibatch_Y})。可以run了之后参数就更新到最优，也可以得到cost了


In [ ]:
#开始时间
start_time = time.clock()
#开始训练
parameters = model(X_train, Y_train, X_test, Y_test)
#结束时间
end_time = time.clock()
#计算时差
print("CPU的执行时间 = " + str(end_time - start_time) + " 秒" )